In [15]:
import dotenv
import os

In [16]:
dotenv.load_dotenv()

True

In [ ]:
os.environ

In [ ]:
!pip install beautifulsoup4 requests

In [2]:
import datetime
import requests  # get stuff from the internet
from bs4 import BeautifulSoup  # parsing  

In [3]:
def get_nemzet_headers(soup):
    return soup.find_all("div", class_="enews-article-offerer-title")

def get_index_headers(soup):
    return soup.find_all("h1")

In [4]:
MAX_TITLES = 10

In [5]:
sites = [
    ("https://index.hu/", get_index_headers),
    ("https://magyarnemzet.hu/", get_nemzet_headers)
]

In [6]:
out = []
for site_url, site_header_getter in sites:

    resp = requests.get(site_url)
    soup = BeautifulSoup(resp.content)
    current_time = datetime.datetime.now().isoformat()
    for i, header in enumerate(site_header_getter(soup)[:MAX_TITLES]):
        a_tag = header.find("a")
        if a_tag is None:
            a_tag = header.find_parent("a")
        
        if a_tag:
            link = a_tag["href"]
        else:
            link = None
        out.append(
            {
                "title": header.text.strip(),
                "link": link, 
                "time": current_time,
                "site": site_url,
                "ind": i + 1,
            }
        )

In [ ]:
!pip install gspread oauth2client

### get oauth credentials [here](https://gspread.readthedocs.io/en/latest/oauth2.html)

### make a spreadsheet, and get id [here](https://drive.google.com/drive/my-drive)

In [8]:
spread_id = "1ti4e4rHKi_6WV4pYccY1YvX0t17dgyXwSBdoiZH2i8M"

In [29]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [9]:
import json

In [10]:
d = json.load(open("creds.json"))

In [12]:
pk = d["private_key"]

In [39]:
scope = ['https://spreadsheets.google.com/feeds']

#credentials = ServiceAccountCredentials.from_json_keyfile_name('ci-creds.json', scope)
credentials = ServiceAccountCredentials.from_json_keyfile_dict(d, scope)

In [40]:
gc = gspread.authorize(credentials)

In [73]:
sheet = gc.open_by_key(spread_id).sheet1

In [43]:
record = out[0]

In [44]:
col_headers = list(record.keys())

In [45]:
col_headers

['title', 'link', 'time', 'site', 'ind']

In [47]:
import string

In [48]:
end_letter = string.ascii_uppercase[len(col_headers)-1]

In [51]:
header_range_name = f"A1:{end_letter}1"

In [52]:
header_range_name

'A1:E1'

In [53]:
# add headres:
sheet.update(header_range_name, [col_headers])

{'spreadsheetId': '1ti4e4rHKi_6WV4pYccY1YvX0t17dgyXwSBdoiZH2i8M',
 'updatedRange': 'Sheet1!A1:E1',
 'updatedRows': 1,
 'updatedColumns': 5,
 'updatedCells': 5}

In [74]:
end_of_sheet = sheet.row_count

In [75]:
end_of_sheet

21

In [76]:
rows = [list(d.values()) for d in out]

In [77]:
sheet.add_rows(len(rows))

In [78]:
sheet_range_name = f"A{end_of_sheet + 1}:{end_letter}{len(rows)+end_of_sheet}"

In [79]:
sheet_range_name

'A22:E41'

In [80]:
sheet.update(
    sheet_range_name,
    rows
)

{'spreadsheetId': '1ti4e4rHKi_6WV4pYccY1YvX0t17dgyXwSBdoiZH2i8M',
 'updatedRange': 'Sheet1!A22:E41',
 'updatedRows': 20,
 'updatedColumns': 5,
 'updatedCells': 100}